In [1]:
from pathlib import Path
import json

data_path = Path("/home/ahurta92/data")

In [2]:
frequency_json = data_path / "frequency.json"
with open(frequency_json) as f:
    frequency = json.load(f)

frequency

{'Ar': {'hf': {'dipole': [0.0,
    0.030119,
    0.060238,
    0.09035699999999999,
    0.120476,
    0.150595,
    0.18071399999999999,
    0.210833,
    0.240952],
   'excited-state': 4}},
 'BF': {'hf': {'dipole': [0.0,
    0.014329875,
    0.02865975,
    0.042989625000000004,
    0.0573195,
    0.071649375,
    0.08597925000000001,
    0.100309125,
    0.114639],
   'excited-state': 4}},
 'BH2Cl': {'hf': {'dipole': [0.0,
    0.0141258125,
    0.028251625,
    0.0423774375,
    0.05650325,
    0.07062906249999999,
    0.084754875,
    0.0988806875,
    0.1130065],
   'excited-state': 4}},
 'BH2F': {'hf': {'dipole': [0.0,
    0.01616975,
    0.0323395,
    0.048509250000000004,
    0.064679,
    0.08084875,
    0.09701850000000001,
    0.11318825,
    0.129358],
   'excited-state': 4}},
 'BH3': {'hf': {'dipole': [0.0,
    0.015157125,
    0.03031425,
    0.045471375,
    0.0606285,
    0.075785625,
    0.09094275,
    0.10609987500000001,
    0.121257],
   'excited-state': 4}},
 'BHF

In [3]:
example_json = Path('/home/ahurta92/data/summer_2024/may2024/output/HF/output/moldft.calc_info.json')
mol_dir = Path('/home/ahurta92/data/summer_2024/may2024/molecules')
with open(example_json) as f:
    example = json.load(f)


In [4]:
example['molecule']

{'geometry': [[0.0, 0.0, -0.09168], [0.0, 0.0, 0.82512]],
 'parameters': {'eprec': 0.0001,
  'field': [0.0, 0.0, 0.0],
  'no_orient': False,
  'psp_calc': False,
  'pure_ae': True,
  'symtol': -0.01},
 'symbols': ['F', 'H']}

In [5]:

class geometry_parameters:
    def __init__(self, eprec=None, field=None, no_orient=None, psp_calc=None, pure_ae=None, symtol=None, core_type=None, units=None):

        self.eprec = 1e-4
        self.field = [0.0, 0.0, 0.0]
        self.no_orient = False
        self.psp_calc = False
        self.pure_ae = True
        self.symtol =-1e-2
        self.core_type = "none"
        self.units = "atomic"
        
        if eprec is not None:
            self.eprec = float(eprec)
        if field is not None:
            self.field = field
        if no_orient is not None:
            self.no_orient = no_orient
        if psp_calc is not None:
            self.psp_calc = psp_calc
        if pure_ae is not None:
            self.pure_ae = pure_ae
        if symtol is not None:
            self.symtol = symtol
        if core_type is not None:
            self.core_type = core_type
        if units is not None:
            self.units = units
            

    def __repr__(self):
        return f"eprec: {self.eprec}, field: {self.field}, no_orient: {self.no_orient}, psp_calc: {self.psp_calc}, pure_ae: {self.pure_ae}, symtol: {self.symtol}, core_type: {self.core_type}, units: {self.units}"
    def __to_json__(self):
        return {"eprec": float(self.eprec), "field": self.field, "no_orient": self.no_orient, "psp_calc": self.psp_calc, "pure_ae": self.pure_ae, "symtol": self.symtol, "core_type": self.core_type, "units": self.units}


class MADNESS_MOLECULE:
    def __init__(self, geometry=None, symbols=None, parameters=None):
        
        self.geometry = []
        self.symbols = []
        self.parameters = geometry_parameters()
        
        if geometry is not None:
            self.geometry = geometry 
        if symbols is not None:
            self.symbols = symbols
        if parameters is not None:
            self.parameters = geometry_parameters(**parameters)

    def from_molfile(self, molfile: Path):
        with open(molfile) as f:
            for line in f:
                if line.startswith("end"):
                    break
                if line and not line.startswith("#"):
                    if line.startswith("geometry"):
                        continue
                    split = line.split()
                    if split[0] in self.parameters.__dict__.keys():
                        self.parameters.__dict__[split[0]] = split[1]
                        
                    else:
                        if len(split) != 4:
                            break
                        else:
                            self.symbols.append(split[0])
                            self.geometry.append([float(i) for i in split[1:]])

    def __to_json__(self):
        return {"geometry": self.geometry, "symbols": self.symbols, "parameters": self.parameters.__to_json__()}
        
    def __repr__(self):
        return f"parameters: {self.parameters}\ngeometry: {self.geometry}\nsymbols: {self.symbols}"


def dict_to_object(obj_class, data):
    return obj_class(**data)


mol = MADNESS_MOLECULE(**example['molecule'])
print(mol)



parameters: eprec: 0.0001, field: [0.0, 0.0, 0.0], no_orient: False, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: atomic
geometry: [[0.0, 0.0, -0.09168], [0.0, 0.0, 0.82512]]
symbols: ['F', 'H']


In [6]:
mol = MADNESS_MOLECULE()
mol.from_molfile(mol_dir / "HF.mol")
print(mol)



parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: False, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.0, 0.0], [0.0, 0.0, 0.916799999037]]
symbols: ['F', 'H']


In [7]:
import glob
mol_json={}
for mol in mol_dir.glob("*.mol"):
    mol_name=mol.name.split('.')[0]
    
    madmol = MADNESS_MOLECULE()
    madmol.from_molfile(mol)
    madmol.parameters.no_orient = True
    print(madmol)
    mol_json[mol_name]=madmol
    
    #mol.parameters.no_orient = True



parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.0, 0.0], [0.0, 0.0, 1.747549999712], [0.987706427326, 0.0, 1.985719104715], [-0.332288780655, -0.930133402042, 1.98571912059]]
symbols: ['Cl', 'N', 'H', 'H']
parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.0, 0.0], [0.0, 0.0, 2.88900000188]]
symbols: ['Li', 'Na']
parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.0, 0.0], [0.0, 0.0, 1.411900001834]]
symbols: ['F', 'F']
parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[1.037589909768, 0.143838202975, 0.0], [-0.543781337495, -0.012698649649, 0

In [8]:
mol_json['N2H4']

parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.723000000685, -0.112299997942], [0.0, -0.723000000685, -0.112299997942], [-0.447000001016, 1.00310000188, 0.756200002079], [0.447000001016, -1.00310000188, 0.756200002079], [0.96630000263, 1.00310000188, 0.030100001918], [-0.96630000263, -1.00310000188, 0.030100001918]]
symbols: ['N', 'N', 'H', 'H', 'H', 'H']

In [9]:
dalton_excited_json = data_path / "dalton-excited.json"


In [10]:
json_data_dir = Path("/home/ahurta92/data/may2024/json_data")
dalton_excited_json = json_data_dir / Path("dalton-excited.json")
with open(dalton_excited_json, 'r') as f:
    dalton_excited = json.load(f)
first_excited_freq = {}
for mol, data, in dalton_excited['hf'].items():
    first_excited_freq[mol] = data['aug-cc-pVTZ']['response']['freq'][0]

num_freq = 4

freq_range = {}
for mol, freq in first_excited_freq.items():
    omega = freq / 4 / num_freq
    omegas = [i * omega for i in range(num_freq + 1)]
    # truncate omegas to 3 decimal places

    freq_range[mol] = omegas


In [11]:
quad_input = Path("/home/ahurta92/dev/devtest/quad_input.json")
with open(quad_input) as f:
    quad = json.load(f)

dft = {}
dft['xc'] = 'hf'
dft['l'] = 200
dft['maxiter'] = 30
dft['localize'] = 'new'
dft['protocol'] = [1e-4, 1e-6]

quad['dft'] = dft


In [12]:
json_input_dir = Path("/home/ahurta92/data/may2024/json_input")
# if it exists remove it 
if json_input_dir.exists():
    for file in json_input_dir.iterdir():
        file.unlink()
json_input_dir.mkdir(exist_ok=True)


In [13]:

base_quad = quad.copy()
for mol in freq_range:
    quad = base_quad.copy()
    quad['molecule'] = mol_json[mol].__to_json__()
    quad['input_name'] = mol
    quad['response']['freq_range'] = freq_range[mol]  # based on the above
    quad['response']['protocol'] = [1e-4, 1e-6]
    with open(json_input_dir / f"{mol}.json", "x") as f:
        json.dump(quad, f, indent=4)

In [14]:
mol_json['HF'].__to_json__()

{'geometry': [[0.0, 0.0, 0.0], [0.0, 0.0, 0.916799999037]],
 'symbols': ['F', 'H'],
 'parameters': {'eprec': 1e-06,
  'field': [0.0, 0.0, 0.0],
  'no_orient': True,
  'psp_calc': False,
  'pure_ae': True,
  'symtol': -0.01,
  'core_type': 'none',
  'units': 'angstrom'}}

In [15]:
mol_json['HF'] 


parameters: eprec: 1e-6, field: [0.0, 0.0, 0.0], no_orient: True, psp_calc: False, pure_ae: True, symtol: -0.01, core_type: none, units: angstrom
geometry: [[0.0, 0.0, 0.0], [0.0, 0.0, 0.916799999037]]
symbols: ['F', 'H']

In [131]:
freq_range['H2O']

[0.0, 0.0198666875, 0.039733375, 0.0596000625, 0.07946675]